# Create product embedding for instacart dataset

Idea: similar to word2vec, products in the same order are close to each other

## Load orders and product name

In [1]:
!ls data

aisles.csv		       order_products__train.csv.zip
aisles.csv.zip		       orders.csv
departments.csv.zip	       orders.csv.zip
__MACOSX		       products.csv
order_products__prior.csv      products.csv.zip
order_products__prior.csv.zip  sample_submission.csv
order_products__train.csv      sample_submission.csv.zip


In [2]:
import pandas as pd

product_df = pd.read_csv("data/products.csv")
order_df = pd.read_csv("data/orders.csv")
order_prior_df = pd.read_csv("data/order_products__prior.csv")
# print(product_df.head(10))
# print(order_df.head(10))
# print(order_prior_df.head(10))



In [3]:
print(f"Number of products: {len(product_df)}")
print(f"Number of orders: {len(order_prior_df['order_id'].value_counts())}")

Number of products: 49688
Number of orders: 3214874


In [4]:
# Filter our order with less than 2 products
df1 = order_prior_df[order_prior_df.groupby("order_id")['order_id'].transform('size') > 80]
print(f"Number of orders with at least 2 products: {len(df1['order_id'].value_counts())}")

# Get list of products in these orders
product_ids = df1["product_id"].unique().tolist()
product_df1 = product_df[product_df["product_id"].isin(product_ids)]
print(f"Number of reduced products: {len(product_ids)}")

Number of orders with at least 2 products: 111
Number of reduced products: 4252


In [5]:
# Build product mapping

product_ids = product_df1["product_id"]
product_names = product_df1["product_name"]
# map product id to indices starting from 0
product_id_to_ind = {product_id: i for i, product_id in enumerate(product_ids)}
product_ind_to_name = {i: product_name for i, product_name in enumerate(product_names)}
product_name_to_ind = {product_name: i for i, product_name in enumerate(product_names)}

In [15]:
# torch dataset
import random
from torch import nn
from torch.utils.data import DataLoader, Dataset
import tqdm

class ProductPairDataset(Dataset):
    """Dataset class that returns a pair of (context, target) product ids.
    
    The pair is a random combination of 2 products in the same order.
    
    """
    def __init__(self, order_df, product_id_to_ind, num_noisy_products: int = 20):
        
        self.id_to_ind = product_id_to_ind
        unique_orders = order_df["order_id"].unique().tolist()
        order_to_ind = {
            order_id: i
            for i, order_id in enumerate(unique_orders)
        }
        # self.orders to store a list of orders, each order is represented by a list of products in that order
        self.orders = [[] for _ in unique_orders]
        for order_id, product_id in tqdm.tqdm(zip(order_df["order_id"], order_df["product_id"])):
            order_ind = order_to_ind[order_id]
            product_ind = product_id_to_ind[product_id]
            self.orders[order_ind].append(product_ind)
        self.num_noisy_products = num_noisy_products
        self.num_products = len(self.id_to_ind)        
            
        
    def __len__(self):
        # The true number of pairs is much higher than this, but it does not really matter
        # as long as we train the model long enough.
        return len(order_df)
    
    def __getitem__(self, index):
        # Get a random order and a random pair of products in that order
        products = random.choice(self.orders)
        selected_products = random.sample(products, 2)
        print(selected_products)
        print(self.num_products)
        context_product_ind = selected_products[0]
        target_product_ind = selected_products[1]
        # buid mask
        mask = [0 for _ in range(self.num_products)]
        random_ids = random.sample(range(self.num_products), self.num_noisy_products)
        for i in random_ids + [context_product_ind]:
            mask[i] = 1
        return context_product_ind, target_product_ind, mask
        
training_data = ProductPairDataset(df1, product_id_to_ind)
  

In [16]:
batch_size = 2
train_dataloader = DataLoader(
    training_data, batch_size=batch_size, shuffle=True, num_workers=2
)     

In [17]:
# Build the pytorch lightning model
import pytorch_lightning as pl
import torch.multiprocessing
import torch.nn.functional as F
class SigmoidBCELoss(nn.Module):
    "BCEWithLogitLoss with masking on call."

    def __init__(self):
        super().__init__()

    def forward(self, inputs, target, mask=None):
        out = nn.functional.binary_cross_entropy_with_logits(
            inputs, target, weight=mask, reduction="none")
        return out.mean(dim=1)

loss_fn = SigmoidBCELoss()
loss2 = torch.nn.CrossEntropyLoss()
class Prod2VecModel(pl.LightningModule):
    def __init__(self, num_products, embed_size: int = 50):
        super().__init__()
        self.embed_size = embed_size
        self.embedding = nn.Embedding(num_products, self.embed_size)
        self.hidden = nn.Linear(self.embed_size, num_products, bias=False)


    def forward(self, contexts):
#         breakpoint()
        hid = self.embedding(contexts)
        pre_sigmoid = self.hidden(hid)
        return pre_sigmoid

    def training_step(self, batch, batch_idx):
        contexts, targets, masks = batch
        output = self.forward(contexts)
        # loss = loss_fn(output, targets, masks)
        loss = loss2(output, targets)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(), lr=0.5, momentum=0.5, weight_decay=1e-3
        )  # learning rate
        return optimizer

In [ ]:
num_products = len(product_id_to_ind)
embed_size = 2
model = Prod2VecModel(num_products, embed_size)
trainer = pl.Trainer(gpus=1, max_epochs=1)
trainer.fit(model, train_dataloader, train_dataloader)

In [10]:
import torch
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()

In [11]:
output

tensor(1.7500, grad_fn=<NllLossBackward>)

In [12]:
input

tensor([[-0.2454, -1.8665, -0.1902, -0.2009, -1.7256],
        [-0.0860,  0.3291,  0.4831, -0.9865, -1.7710],
        [-1.9306,  0.2695, -0.2671,  0.2023,  0.8686]], requires_grad=True)

In [13]:
target

tensor([2, 3, 3])

In [14]:
a = [[] for i in range(3)]
print(a)

[[], [], []]


In [15]:
a[0].append(1)
a[1].append(2)
print(a)

[[1], [2], []]


In [16]:
product_df1.iloc[0]["product_name"]

'Chocolate Sandwich Cookies'